# Brain

In [2]:
setwd('/gpfs/gibbs/pi/zhao/xs272/prep_pipeline/')
library(Matrix)
library(readr)
source('/gpfs/gibbs/pi/zhao/xs272/data_preprocess/utils.r')
source('/gpfs/gibbs/pi/zhao/xs272/Multiomics/sc_benchmark/scripts/utils.r')

Attaching SeuratObject

Attaching sp

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    

## 1. Unimodal RNA preprocessing
FACS

In [ ]:
smart.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/FACS'
smart.rna.m <- read.csv(file.path(smart.path, 'Brain_Myeloid-counts.csv'))

smart.rna.mat.m <- as(data.matrix(smart.rna.m[, -1]), 'dgCMatrix')

rownames(smart.rna.mat.m) <- format.genes(smart.rna.m[, 1])
colnames(smart.rna.mat.m) <- colnames(smart.rna.m)[-1]

In [ ]:
smart.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/FACS'
smart.rna.nm <- read.csv(file.path(smart.path, 'Brain_Non-Myeloid-counts.csv'))

smart.rna.mat.nm <- as(data.matrix(smart.rna.nm[, -1]), 'dgCMatrix')

rownames(smart.rna.mat.nm) <- format.genes(smart.rna.nm[, 1])
colnames(smart.rna.mat.nm) <- colnames(smart.rna.nm)[-1]

In [ ]:
identical(rownames(smart.rna.mat.m), rownames(smart.rna.mat.nm))

In [ ]:
any(duplicated(colnames(smart.rna.mat.m), colnames(smart.rna.mat.nm)))

In [ ]:
smart.rna.mat <- cbind(smart.rna.mat.m, smart.rna.mat.nm)

In [ ]:
print.count.dim(smart.rna.mat.m)
print.count.dim(smart.rna.mat.nm)
print.count.dim(smart.rna.mat)

In [ ]:
facs.anno.file <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/RNA/annotations_facs_brain.csv'
facs.anno.table <- read.csv(facs.anno.file)

In [ ]:
facs.count <- smart.rna.mat
print.names.match(colnames(facs.count), facs.anno.table$cell)

In [ ]:
facs.count <- facs.count[, facs.anno.table$cell]
facs.anno <- facs.anno.table$annotation

In [ ]:
print.count.dim(facs.count)

## 2.  Unimodal gene activity (sci-ATAC-seq)

In [ ]:
sci.ga.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC'
sci.ga.mat <- readMM(file.path(sci.ga.path, 'activity_scores.quantitative.mtx.gz'))
sci.ga.genes <- readLines(file.path(sci.ga.path, 'activity_scores.quantitative.genes.txt'))
sci.ga.cells <- readLines(file.path(sci.ga.path, 'activity_scores.quantitative.cells.txt'))

In [ ]:
any(duplicated(format.genes(sci.ga.genes)))

In [ ]:
sci.ga.mat <- set.names(counts = sci.ga.mat, rowname = format.genes(sci.ga.genes), colname = sci.ga.cells)

In [ ]:
sci.ga.annot <- read.csv(file.path(sci.ga.path, 'cell_metadata.tissue_freq_filtered_brain.txt'))

In [ ]:
any(duplicated(sci.ga.annot$cell))

In [ ]:
unique(sci.ga.annot$tissue)

In [ ]:
print.names.match(sci.ga.annot$cell, colnames(sci.ga.mat))

In [19]:
sci.ga.mat <- sci.ga.mat[, sci.ga.annot$cell]

## 3. Align unimodal RNA and ATAC gene activities
gene names  
cell types  

In [20]:
print.names.match(rownames(facs.count), rownames(sci.ga.mat))

[1] "Length1, Length2, Length intersection:"
[1] "23433 20783 17165"


In [21]:
sort(unique(facs.anno))

[1] "Astrocyte"        "Endothelial cell" "Macrophage"       "Microglia"       
[5] "Neuron"           "Oligodendrocyte"  "OPC"              "Pericyte"

In [22]:
sort(unique(sci.ga.annot$annotation))

[1] "Astrocyte"        "Collision"        "Endothelial cell" "Microglia"       
[5] "Neuron"           "Oligodendrocyte"  "Unknown"

In [23]:
print.names.match(unique(facs.anno), unique(sci.ga.annot$annotation))

[1] "Length1, Length2, Length intersection:"
[1] "8 7 5"


In [24]:
facs.path <- 'brain/FACS/'
write.counts.transpose(facs.count, facs.path)
write.annotation(facs.anno, facs.path)

Warning message in dir.create(folder, recursive = TRUE):
"'brain/FACS' already exists"
Warning message in dir.create(folder, recursive = TRUE):
"'brain/FACS' already exists"


In [25]:
sci.ga.path <- 'brain/sci-atac-gene-activities/'
write.counts.transpose(sci.ga.mat, sci.ga.path)
write.annotation(sci.ga.annot$annotation, sci.ga.path)

Warning message in dir.create(folder, recursive = TRUE):
"'brain/sci-atac_gene_activities' already exists"
Warning message in dir.create(folder, recursive = TRUE):
"'brain/sci-atac_gene_activities' already exists"


## 4. Unimodal ATAC (sci-ATAC-seq)

In [26]:
atac.peak.path = 'tmp/sci_atac'
system(paste0('mkdir -p ', atac.peak.path))
atac.bin.path <- "/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC/atac_matrix.binary.qc_filtered.mtx.gz"

In [27]:
system(paste0('bash bin2int_mtx.sh -z -i ', atac.bin.path, ' -o ', atac.peak.path, '/counts_int.mtx'))

In [ ]:
sci.atac.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC/'
sci.atac.counts <- readMM(file.path(atac.peak.path, 'counts_int.mtx'))
sci.atac.barcodes <- readLines(file.path(sci.atac.path, 'atac_matrix.binary.qc_filtered.cells.txt'))
sci.atac.peaks <- readLines(file.path(sci.atac.path, 'atac_matrix.binary.qc_filtered.peaks.txt'))

In [ ]:
sci.atac.counts <- set.names(counts = sci.atac.counts, rowname = format.peaks(sci.atac.peaks), colname = sci.atac.barcodes)

In [ ]:
sci.atac.annot <- read.csv(file.path(sci.atac.path, 'cell_metadata.tissue_freq_filtered_brain.txt'))

In [ ]:
any(duplicated(sci.atac.annot$cell))

In [ ]:
unique(sci.atac.annot$tissue)

In [ ]:
print.names.match(sci.atac.annot$cell, colnames(sci.atac.counts))

In [ ]:
sci.atac.counts <- sci.atac.counts[, sci.atac.annot$cell]

In [ ]:
identical(colnames(sci.atac.counts), colnames(sci.ga.mat))

In [ ]:
sci.atac.path <- 'brain/sci-atac'
write.counts.transpose(sci.atac.counts, sci.atac.path)
write.annotation(sci.atac.annot$annotation, sci.atac.path)

## 5. Unimodal ATAC fragment file

In [48]:
frag.path <- "/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC/fragment_mm10"
frag.files <- c(
                Cerebellum_62216 = "Cerebellum_62216.split.q10.fragments.sort.bed.gz",
                WholeBrainA_62216 = "WholeBrainA_62216.split.q10.fragments.sort.bed.gz",
                PreFrontalCortex_62216 = "PreFrontalCortex_62216.split.q10.fragments.sort.bed.gz",
                WholeBrainA_62816 = "WholeBrainA_62816.split.q10.fragments.sort.bed.gz"
                )

In [ ]:
# need to change space to \t? first do not change, see if it works.
# system(paste('bash frag_prep.sh -i', infile, '-o', outfolder, '-z', '-s', sep=' '))

## 6. Multi ATAC preprocess: fix peaks

### 6.1 SHARE-seq

In [40]:
share.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/SHARE-seq/brain/'
# format peak names.
system('mkdir -p brain/share/tmp/ATAC')
system(paste('zcat', file.path(share.path, 'GSM4156599_brain.peaks.bed.gz'), '| sed -E \'s/\\s+/-/g\' > brain/share/tmp/ATAC/peaks.txt', sep=' '))

In [41]:
share.atac.counts <- readMM(file.path(share.path, 'GSM4156599_brain.counts.txt.gz'))
share.atac.barcodes <- readLines(file.path(share.path, 'GSM4156599_brain.barcodes.txt.gz'))
share.atac.peaks <- readLines('brain/share/tmp/ATAC/peaks.txt')
share.atac.barcodes <- format.barcode(share.atac.barcodes)

In [42]:
any(duplicated(share.atac.barcodes))

[1] FALSE

In [43]:
share.atac.counts <- set.names(counts = share.atac.counts, rowname = share.atac.peaks, colname = share.atac.barcodes)

### 6.2 SNARE-seq

In [14]:
snare.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/SNARE-seq/ATAC'
# format peak names.
system('mkdir -p brain/snare/tmp/ATAC')
system(paste('zcat', file.path(snare.path, 'ATAC', 'GSE126074_AdBrainCortex_SNAREseq_chromatin.peaks.tsv.gz'), '| sed -E \'s/\\s+|:/-/g\' > brain/snare/tmp/ATAC/peaks.txt', sep=' ')) # change : to -

In [5]:
snare.atac.counts <- readMM(file.path(snare.path, 'GSE126074_AdBrainCortex_SNAREseq_chromatin.counts.mtx.gz'))
snare.atac.barcodes <- readLines(file.path(snare.path, 'GSE126074_AdBrainCortex_SNAREseq_chromatin.barcodes.tsv.gz'))
snare.atac.peaks <- readLines('brain/snare/tmp/ATAC/peaks.txt')

In [6]:
any(duplicated(snare.atac.barcodes))

[1] FALSE

In [11]:
snare.atac.counts <- set.names(counts = snare.atac.counts, rowname = snare.atac.peaks, colname = snare.atac.barcodes)

## 7. Multi RNA preprocess

### 7.1 SHARE-seq

In [ ]:
share.rna.counts <- read.table(gzfile(file.path(share.path, 'GSM4156610_brain.rna.counts.txt.gz')), header=TRUE)
rownames(share.rna.counts) <- share.rna.counts$gene
share.rna.counts <- as(as.matrix(share.rna.counts[, -1]), 'dgCMatrix')
rownames(share.rna.counts) <- format.genes(rownames(share.rna.counts))
colnames(share.rna.counts) <- format.barcode(colnames(share.rna.counts))

#### Check if feature sets have large overlap with single RNA

In [11]:
# facs.path <- 'brain/FACS/'
# facs.count <- load.rna.folder(facs.path)

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


In [12]:
print.names.match(rownames(share.rna.counts), rownames(facs.count))

[1] "Length1, Length2, Length intersection:"
[1] "21127 23433 16304"


### 7.2 SNARE-seq

In [24]:
snare.rna.counts <- readMM(file.path(snare.path, 'RNA' ,'GSE126074_AdBrainCortex_SNAREseq_cDNA.counts.mtx.gz'))
snare.rna.barcode <- readLines(file.path(snare.path, 'RNA', 'GSE126074_AdBrainCortex_SNAREseq_cDNA.barcodes.tsv.gz'))
snare.rna.gene <- readLines(file.path(snare.path, 'RNA', 'GSE126074_AdBrainCortex_SNAREseq_cDNA.genes.tsv.gz'))

In [25]:
any(duplicated(format.genes(snare.rna.gene)))

[1] TRUE

In [26]:
dupl.id <- (1:length(snare.rna.gene))[duplicated(format.genes(snare.rna.gene))]

In [29]:
snare.rna.gene[dupl.id]

[1] "PISD-PS1"

In [27]:
# drop duplicated gene name.
snare.rna.gene <- format.genes(snare.rna.gene[-dupl.id])
snare.rna.counts <- snare.rna.counts[-dupl.id, ]

In [28]:
snare.rna.counts <- set.names(counts = snare.rna.counts, rowname = format.genes(snare.rna.gene), colname = format.barcode(snare.rna.barcode))

In [30]:
facs.path <- 'brain/FACS/'
facs.count <- load.rna.folder(facs.path)

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


In [31]:
print.names.match(rownames(snare.rna.counts), rownames(facs.count))

[1] "Length1, Length2, Length intersection:"
[1] "33159 23433 18323"


## 8. Align multi ATAC RNA

trans table is for frag barcode remapping, no need for rna.  
but need to make sure they contain the same cells.

### 8.1. SHARE-seq

In [56]:
# trans.tab <- read.table(gzfile(file.path(share.path, 'GSM4156599_brain_celltype.txt.gz')), header=TRUE)
# trans.tab$atac.bc <- format.barcode(trans.tab$atac.bc)
# trans.tab$rna.bc <- format.barcode(trans.tab$rna.bc)

In [9]:
print.names.match(colnames(share.rna.counts), colnames(share.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "5875 3293 3293"


In [10]:
share.rna.counts <- share.rna.counts[, colnames(share.atac.counts)]

In [11]:
identical(colnames(share.rna.counts), colnames(share.atac.counts))

[1] TRUE

### 8.2. SNARE-seq

In [32]:
print.names.match(colnames(snare.rna.counts), colnames(snare.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "10309 10309 10309"


In [33]:
snare.rna.counts <- snare.rna.counts[, colnames(snare.atac.counts)]

In [34]:
identical(colnames(snare.rna.counts), colnames(snare.atac.counts))

[1] TRUE

## 9. Map unimodal ATAC peaks to multi ATAC peaks

In [35]:
# load results of previous steps because the program quitted.
sci.atac.data <- load.data.folder('brain/sci-atac')

sci.atac.counts <- sci.atac.data$counts
sci.atac.annota <- sci.atac.data$annotations

atac.peak.path = 'tmp/sci_atac'
sci.atac.path <- '/gpfs/gibbs/pi/zhao/yw599/Multiome/data/MouseAtlas/ATAC/'
sci.atac.annot <- read.csv(file.path(sci.atac.path, 'cell_metadata.tissue_freq_filtered_brain.txt'))

ann <- (sci.atac.annota)
names(ann) <- NULL
identical(ann, sci.atac.annot$annotation)

[1] TRUE

In [36]:
unique(sci.atac.annot$tissue.replicate)

[1] "WholeBrainA_62816"      "Cerebellum_62216"       "WholeBrainA_62216"     
[4] "PreFrontalCortex_62216"

In [38]:
library(EnsDb.Mmusculus.v79) # mm10
library(Signac)
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

### 9.1. SHARE-seq

In [ ]:
mapped.atac.counts.list <- list()
for (tissue in names(frag.files)) {
    selected.id <- which(sci.atac.annot$tissue.replicate == tissue)
    selected.sci.count <- sci.atac.counts[, selected.id]
    selected.frag.path <- file.path(frag.path, frag.files[tissue])
    mapped.atac.counts.list[[tissue]] <- map.peaks(selected.sci.count, share.atac.counts, selected.frag.path, annotations)
}

In [127]:
# taking intersection of peaks among tissues
intersect.peaks <- rownames(mapped.atac.counts.list[[1]])
for (i in 2:length(mapped.atac.counts.list)) {
    intersect.peaks <- intersect(intersect.peaks, rownames(mapped.atac.counts.list[[i]]))
}

for (tissue in names(mapped.atac.counts.list)) {
    mapped.atac.counts.list[[tissue]] <- mapped.atac.counts.list[[tissue]][intersect.peaks ,]
}

mapped.sci.atac.counts <- do.call(cbind, mapped.atac.counts.list)

In [ ]:
identical(rownames(mapped.sci.atac.counts), rownames(share.atac.counts))

### 9.2. SNARE-seq

In [ ]:
snare.mapped.atac.counts.list <- list()
for (tissue in names(frag.files)) {
    selected.id <- which(sci.atac.annot$tissue.replicate == tissue)
    selected.sci.count <- sci.atac.counts[, selected.id]
    selected.frag.path <- file.path(frag.path, frag.files[tissue])
    snare.mapped.atac.counts.list[[tissue]] <- map.peaks(selected.sci.count, snare.atac.counts, selected.frag.path, annotations)
}

In [51]:
# taking intersection of peaks among tissues
intersect.peaks <- rownames(snare.mapped.atac.counts.list[[1]])
for (i in 2:length(snare.mapped.atac.counts.list)) {
    intersect.peaks <- intersect(intersect.peaks, rownames(snare.mapped.atac.counts.list[[i]]))
}

for (tissue in names(snare.mapped.atac.counts.list)) {
    snare.mapped.atac.counts.list[[tissue]] <- snare.mapped.atac.counts.list[[tissue]][intersect.peaks ,]
}

snare.mapped.sci.atac.counts <- do.call(cbind, snare.mapped.atac.counts.list)

In [52]:
identical(rownames(snare.mapped.sci.atac.counts), rownames(snare.atac.counts))

[1] FALSE

In [53]:
print.names.match(rownames(snare.mapped.sci.atac.counts), rownames(snare.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "134968 244544 134968"


## 10. Preprocess unimodal atac matched

### 10.1 Mapped to SHARE-seq

In [135]:
print.names.match(sci.atac.annot$cell, colnames(mapped.sci.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "16874 16874 16874"


In [134]:
identical(sci.atac.annot$cell, colnames(mapped.sci.atac.counts))

[1] FALSE

In [138]:
# reorder by annotation file, same as atac before matching to multi peaks.
mapped.sci.atac.counts <- mapped.sci.atac.counts[, sci.atac.annot$cell]

### 10.2 Mapped to SNARE-seq

In [54]:
print.names.match(sci.atac.annot$cell, colnames(snare.mapped.sci.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "16874 16874 16874"


In [55]:
identical(sci.atac.annot$cell, colnames(snare.mapped.sci.atac.counts))

[1] FALSE

In [56]:
# reorder by annotation file, same as atac before matching to multi peaks.
snare.mapped.sci.atac.counts <- snare.mapped.sci.atac.counts[, sci.atac.annot$cell]

In [58]:
identical(sci.atac.annot$cell, colnames(snare.mapped.sci.atac.counts))

[1] TRUE

## 11. Align unimodal and multi ATAC

### 11.1 SHARE-seq

In [146]:
print.names.match(rownames(mapped.sci.atac.counts), rownames(share.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "236580 428041 236580"


In [147]:
share.atac.counts <- share.atac.counts[rownames(mapped.sci.atac.counts) ,]

save data

In [148]:
# single ATAC rematch
write.counts.transpose(mapped.sci.atac.counts, 'brain/sci-atac-mapped-peaks-to-share')
write.annotation(sci.atac.annot$annotation, 'brain/sci-atac-mapped-peaks-to-share')
# share RNA
write.counts.transpose(share.rna.counts, 'brain/share/RNA/')
# share ATAC
write.counts.transpose(share.atac.counts, 'brain/share/ATAC/')

Warning message in dir.create(folder, recursive = TRUE):
"'brain/sci-atac-mapped-peaks' already exists"


### 11.2 SNARE-seq

In [59]:
print.names.match(rownames(snare.mapped.sci.atac.counts), rownames(snare.atac.counts))

[1] "Length1, Length2, Length intersection:"
[1] "134968 244544 134968"


In [60]:
snare.atac.counts <- snare.atac.counts[rownames(snare.mapped.sci.atac.counts) ,]

In [61]:
# single ATAC rematch
write.counts.transpose(snare.mapped.sci.atac.counts, 'brain/sci-atac-mapped-peaks-to-snare')
write.annotation(sci.atac.annot$annotation, 'brain/sci-atac-mapped-peaks-to-snare')
# share RNA
write.counts.transpose(snare.rna.counts, 'brain/snare/RNA/')
# share ATAC
write.counts.transpose(snare.atac.counts, 'brain/snare/ATAC/')